# impoort Libraries

In [ ]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
%matplotlib inline
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import resnet18
import random
warnings.filterwarnings("ignore", category=UserWarning)

# Loading of CIFAR-10 Dataset with Data Transformations

In [ ]:
# Define label noise levels
noise_levels = [0.1, 0.3, 0.5, 0.8, 0.9]

# Define class names
class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer',
    'dog', 'frog', 'horse', 'ship', 'truck'
]

# Define transformations
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)


100%|██████████| 170498071/170498071 [00:02<00:00, 84531027.31it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#Implementation of Label Noise Functions

In [ ]:
# Function to apply symmetric label noise
def symmetric_label_noise(labels, epsilon):
    noisy_labels = []
    for label in labels:
        if random.random() < epsilon:
            noisy_label = random.choice([i for i in range(10) if i != label])  # Choose a random label different from the correct one
            noisy_labels.append(noisy_label)
        else:
            noisy_labels.append(label)
    return noisy_labels

def asymmetric_label_noise( labels, epsilon):
    noisy_labels = []
    for label in labels:
        if label == 9:  # Check if the label is 'bird'
            if random.random() < epsilon:
                noisy_label = 0  # Flip 'bird' to 'airplane'
            else:
                noisy_label = label
        elif label == 2:  # Check if the label is 'deer'
            if random.random() < epsilon:
                noisy_label = 7  # Flip 'deer' to 'horse'
            else:
                noisy_label = label
        elif label == 3:  # Check if the label is 'cat'
            if random.random() < epsilon:
                noisy_label = 5  # Flip 'cat' to 'dog'
            else:
                noisy_label = label
        elif label == 5:  # Check if the label is 'dog'
            if random.random() < epsilon:
                noisy_label = 3  # Flip 'dog' to 'cat'
            else:
                noisy_label = label
        elif label == 7:  # Check if the label is 'truck'
            if random.random() < epsilon:
                noisy_label = 1  # Flip 'truck' to 'automobile'
            else:
                noisy_label = label
        else:
            noisy_label = label  # For other classes, keep the label as is
        noisy_labels.append(noisy_label)

    noisy_labels = torch.tensor(noisy_labels) # Convert list to tensor
    return noisy_labels


#"Model Training and Testing Functions with Label Noise Handling

In [ ]:
def train_model(train_loader, model, criterion, optimizer, noise_fn, epsilon):
    model.train()  # Set the model to training mode
    running_loss = 0.0  # Initialize the running loss
    correct = 0  # Count of correctly predicted labels
    total = 0  # Total number of labels

    for data, labels in train_loader:  # Iterate over the training data
        optimizer.zero_grad()  # Reset gradients to zero
        data, labels = data.cuda(), labels.cuda()  # Move data and labels to GPU
        noisy_labels = noise_fn(labels.tolist(), epsilon)  # Apply label noise

        outputs = model(data)  # Forward pass: compute predicted outputs
        loss = criterion(outputs, torch.tensor(noisy_labels).cuda())  # Calculate loss
        loss.backward()  # Backward pass: compute gradient
        optimizer.step()  # Perform a single optimization step

        running_loss += loss.item()  # Update running loss
        _, predicted = outputs.max(1)  # Get the predictions
        total += labels.size(0)  # Update the total number of labels
        correct += predicted.eq(torch.tensor(noisy_labels).cuda()).sum().item()  # Update correct predictions

    train_loss = running_loss / len(train_loader)  # Calculate average loss
    train_accuracy = 100 * correct / total  # Calculate training accuracy
    return train_loss, train_accuracy

def test_model(test_loader, model):
    model.eval()  # Set the model to evaluation mode
    correct = 0  # Count of correctly predicted labels
    total = 0  # Total number of labels

    with torch.no_grad():  # Disable gradient calculations
        for data, labels in test_loader:  # Iterate over the test data
            data, labels = data.cuda(), labels.cuda()  # Move data and labels to GPU
            outputs = model(data)  # Forward pass: compute predicted outputs
            _, predicted = outputs.max(1)  # Get the predictions
            total += labels.size(0)  # Update the total number of labels
            correct += predicted.eq(labels).sum().item()  # Update correct predictions

    test_accuracy = 100 * correct / total  # Calculate test accuracy
    return test_accuracy


#Training and Testing Loop for Model with Symmetric and Asymmetric Label Noise"

In [ ]:
import time

for _ in range(10):
    # Training loop for symmetric and asymmetric noise levels
    for epsilon in noise_levels:
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

        # Symmetric Noise Training
        model = resnet18(pretrained=False, num_classes=10).cuda()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.004)
        epochs = 10

        start_train_time = time.time()
        total_train_accuracy_symmetric = 0
        for epoch in range(epochs):
            train_loss, train_accuracy = train_model(train_loader, model, criterion, optimizer, symmetric_label_noise, epsilon)
            total_train_accuracy_symmetric += train_accuracy
        end_train_time = time.time()
        training_time_symmetric = end_train_time - start_train_time

        average_train_accuracy_symmetric = total_train_accuracy_symmetric / epochs

        start_test_time = time.time()
        test_accuracy_symmetric = test_model(test_loader, model)
        end_test_time = time.time()
        inference_time_symmetric = end_test_time - start_test_time

        print(f"Symmetric Noise, Epsilon={epsilon}: Average Train Accuracy: {average_train_accuracy_symmetric:.2f}%, Test Accuracy: {test_accuracy_symmetric:.2f}%, Training Time: {training_time_symmetric:.2f}s, Inference Time: {(inference_time_symmetric/1000):.5f}s")

        # Asymmetric Noise Training
        model = resnet18(pretrained=False, num_classes=10).cuda()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.004)

        start_train_time = time.time()
        total_train_accuracy_asymmetric = 0
        for epoch in range(epochs):
            train_loss, train_accuracy = train_model(train_loader, model, criterion, optimizer, asymmetric_label_noise, epsilon)
            total_train_accuracy_asymmetric += train_accuracy
        end_train_time = time.time()
        training_time_asymmetric = end_train_time - start_train_time

        average_train_accuracy_asymmetric = total_train_accuracy_asymmetric / epochs

        start_test_time = time.time()
        test_accuracy_asymmetric = test_model(test_loader, model)
        end_test_time = time.time()
        inference_time_asymmetric = end_test_time - start_test_time


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/10: Train Loss: 1.6490 | Train Accuracy: 44.57%
Epoch 2/10: Train Loss: 1.3680 | Train Accuracy: 57.01%
Epoch 3/10: Train Loss: 1.2415 | Train Accuracy: 62.26%
Epoch 4/10: Train Loss: 1.1563 | Train Accuracy: 65.51%
Epoch 5/10: Train Loss: 1.1085 | Train Accuracy: 67.65%
Epoch 6/10: Train Loss: 1.0690 | Train Accuracy: 69.72%
Epoch 7/10: Train Loss: 1.0118 | Train Accuracy: 71.42%


KeyboardInterrupt: ignored